### Import libraries

In [432]:
import pandas as pd
from faker import Faker
import numpy as np
import time
import sqlite3
from datetime import datetime
from typing import Callable, Any

### Create generate_ssn function

In [419]:
def generate_ssn(
    num_of_records: int
) -> pd.Series:
    
    """
     Returns Series object contains PESEL numbers.
     Number of records is defined by parameter. 
    
    """
    faker_pl = Faker('pl_PL')
    pesel_list = []

    for num in range(num_of_records):
        pesel_list.append(faker_pl.ssn())

    pesel_arr = np.array(pesel_list)

    pesel_series = pd.Series(pesel_arr)
    pesel_series.index +=1

    return pesel_series


generate_ssn(10)


1     02300484917
2     00321254096
3     89022258644
4     93080450522
5     90022641716
6     17282159254
7     93060594899
8     82101075882
9     02282916435
10    84061734403
dtype: object

### Create generate_unique_ssns function

In [ ]:
# It has the form YYMMDDZZZXQ, where YYMMDD is the date of birth (with century encoded in month field), ZZZ is the personal identification number,
#  X denotes sex (even for females, odd for males) and Q is a parity number.

def generate_unique_ssns(
    num_rec: int,
    gender: str,
    range_date: str
) -> pd.Series:

    """
    Returns Series object contains pesel numbers according to
    gender and date range given as parameters.

    We assume format of range_date as var = 'yyyymmdd, yyyymmdd'.
    We also assume gender must equals to == 'male' or 'female'.

    """
    months_for_19 = [str(x) for x in range(81,93)]
    months_for_20 = ['%.2d' % x for x in range(0, 13)]
    months_for_21 = [str(x) for x in range(21, 33)]

    even_numbers = [2,4,6,8]
    odd_numbers = [1,3,5,7,9]

    faker_pl = Faker('pl_PL')

    pesel_list_female = []
    pesel_list_male = []  

    # while len(pesel_list_female) == num_rec or \
    #     len(pesel_list_male) == num_rec:
    run = True

    while run == True:
        number_pesel = faker_pl.ssn()
        day = str(number_pesel[4:6])
        year = number_pesel[:2]
        new_date = ''

        if number_pesel[2:4] in months_for_19:
            new_date += year +'/'

            if int(number_pesel[2:4]) >= 90:
                new_month = str(1) + str(number_pesel[3])
                new_date += new_month+'/'
                new_date += day
            
            elif int(number_pesel[2:4]) < 90:
                new_month = str(0) + str(number_pesel[3])
                new_date += new_month+'/'
                new_date += day

        elif number_pesel[2:4] in months_for_21:
            new_date += year +'/'
            
            if int(number_pesel[2:4]) >= 30:
                new_month = str(1) + str(number_pesel[3])
                new_date += new_month+'/'
                new_date += day
            
            elif int(number_pesel[2:4]) < 30:
                new_month = str(0) + str(number_pesel[3])
                new_date += new_month+'/'
                new_date += day
        else:
            new_month = str(number_pesel[2:4])
            new_date += year+ '/'+new_month +'/'
            new_date += day
        
    

        
        date_time_obj = datetime.strptime(new_date, '%y/%m/%d')
        
        start_date = range_date[:2] +'/'+ range_date[2:4] +'/' \
                     + range_date[4:6]

        start_date_obj = datetime.strptime(start_date, '%y/%m/%d')
        end_date = range_date[8:10] + '/' + range_date[10:12] + '/' \
                   + range_date[-2:]
        end_date_obj = datetime.strptime(end_date, '%y/%m/%d')
    
        if date_time_obj <= end_date_obj \
            and date_time_obj >= start_date_obj:
            
            if int(number_pesel[-2]) in even_numbers:
                pesel_list_female.append(number_pesel)
                if len(pesel_list_female) == num_rec and gender == 'female':
                    break

            if int(number_pesel[-2]) in odd_numbers:
                pesel_list_male.append(number_pesel)
                if len(pesel_list_male) == num_rec and gender == 'male':
                    break
                
    if gender == 'female':
        pesel_list_female = np.array(pesel_list_female)
        pesel_series = pd.Series(pesel_list_female)
        
    elif gender == 'male':
        pesel_list_male = np.array(pesel_list_male)
        pesel_series = pd.Series(pesel_list_male)


    return pesel_series

0      90010354897
1      90011294051
2      90011842858
3      90010608255
4      90011103179
          ...     
995    90010760139
996    90010506539
997    90010317874
998    90010380397
999    90011043877
Length: 1000, dtype: object


### Count performance of created functions given for given number 1 000, 10 000 i 100 000

In [421]:
def count_performance_generate_ssns(
    function: Callable,
    rec_par:int
    ):
    """
    Count performance of generate_ssns.
    Returns time.
    """
    
    t_start = time.process_time()
    function(rec_par)
    elapsed_time = time.process_time() - t_start
    print('Time performance of ' +str(function) ,'for ' + str(rec_par) \
          + ' records is equal to ' + str(elapsed_time))


In [423]:
def count_performance_generate_unique_ssns(
    function: Callable,
    num_rec: int,
    gender: str,
    range_date: str,
    ):
    
    """
    Function takes the same assumptions like the tested function.
    """

    t_start = time.process_time()
    function(num_rec, gender, range_date)
    elapsed_time = time.process_time() - t_start
    return f'Time performance of {function} for {num_rec} is \
             equal to{elapsed_time}'


In [422]:
count_performance_generate_ssns(generate_ssn, 1000)
count_performance_generate_ssns(generate_ssn, 10000)
count_performance_generate_ssns(generate_ssn, 100000)

Time performance of <function generate_ssn at 0x7faf029d8d30> for 1000 records is equal to 0.03945665800000597
Time performance of <function generate_ssn at 0x7faf029d8d30> for 10000 records is equal to 0.23761146900000085
Time performance of <function generate_ssn at 0x7faf029d8d30> for 100000 records is equal to 2.1333475019999923


In [430]:
count_performance_generate_unique_ssns(generate_unique_ssns, 1000, 'female', '900101, 900119' )
count_performance_generate_unique_ssns(generate_unique_ssns, 10000, 'female', '900101, 900119' )
count_performance_generate_unique_ssns(generate_unique_ssns, 100000, 'female', '900101, 900119' )

'Time performance of <function generate_unique_ssns at 0x7faf01463b80> for 1000              is equal to115.79038496900012'

### Create validate_ssn

In [482]:
pesel = '00271704012'

def validate_ssn(
    number_pesel: str,
    expect_gen: str,
    data_of_birth: str
    ) -> Any:
    """
    Function returns boolean object, depends of provided parameters.
    We assume that data format will be yy/mm/dd.
    """
    answer = True

    even_numbers = [2,4,6,8]
    odd_numbers = [1,3,5,7,9]

    if expect_gen == 'male' and not int(number_pesel[-2]) in odd_numbers:
        answer = False

    if expect_gen == 'female' and not int(number_pesel[-2]) in even_numbers:
        answer = False 


    months_for_19 = [str(x) for x in range(81,93)]
    months_for_21 = [str(x) for x in range(21, 33)]

    
    day = str(number_pesel[4:6])
    year = number_pesel[:2]
    new_date = ''

    if number_pesel[2:4] in months_for_19:
        new_date += year +'/'

        if int(number_pesel[2:4]) >= 90:
            new_month = str(1) + str(number_pesel[3])
            new_date += new_month+'/'
            new_date += day
        
        elif int(number_pesel[2:4]) < 90:
            new_month = str(0) + str(number_pesel[3])
            new_date += new_month+'/'
            new_date += day

    elif number_pesel[2:4] in months_for_21:
        new_date += year +'/'
        
        if int(number_pesel[2:4]) >= 30:
            new_month = str(1) + str(number_pesel[3])
            new_date += new_month+'/'
            new_date += day
        
        elif int(number_pesel[2:4]) < 30:
            new_month = str(0) + str(number_pesel[3])
            new_date += new_month+'/'
            new_date += day
    else:
        new_month = str(number_pesel[2:4])
        new_date += year+ '/'+new_month +'/'
        new_date += day
    
    if new_date != data_of_birth:
        answer = False

    return answer



print(validate_ssn('90011842858', 'male', '90/01/18'))
print(validate_ssn('90011103179', 'male', '90/01/11'))
print(validate_ssn('90010608255', 'male', '90/01/06'))
print(validate_ssn('02211008561', 'female','02/01/10'))
print(validate_ssn('00271704012', 'male', '00/07/17'))

True
True
True
True
True


### Create database and connection

In [22]:
# Create database
con = sqlite3.connect('flight.db')

# Create cursor in order to permorm SQL commands
cur = con.cursor()

# Create FlightLeg table with specified columns
cur.execute(
    '''
    CREATE TABLE FlightLeg ( 
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    tailNumber text,
    source_airport_code INTEGER,
    source_country_code text,
    destination_airport_code text,
    destination_country_code text,
    departure_time text,
    landing_time text
    )
    ''')

con.close()

### Insert data from csv file

In [24]:
con = sqlite3.connect('flight.db')

cur = con.cursor()

flight_df = pd.read_csv('flightlegs.csv', delimiter=';')\
              .to_sql('temporary_table2', con)

cur.execute(
    '''
    INSERT INTO FlightLeg 
    SELECT *
    FROM temporary_table2
    '''
    )

con.commit()

con.close()


### Create two columns and update their values

In [57]:
con = sqlite3.connect('flight.db')

cur = con.cursor()

# Add flightDuration column
cur.execute(
    '''
    ALTER TABLE FlightLeg 
    ADD COLUMN flightDuration INTEGER
    '''
    )

# Add flightType column
cur.execute(
    '''
    ALTER TABLE FlightLeg
    ADD COLUMN flightType INTEGER
    '''
    )

# Count duration of flight in minutes 
cur.execute(
    '''
    UPDATE FlightLeg 
    SET flightDuration = 
    CAST(
    ROUND(((
    JULIANDAY(landing_time)
    - JULIANDAY(departure_time)) *86400 )/60) as  INT  )
    '''
    )

# Make flightType column out with values 
cur.execute(
    '''
    UPDATE FlightLeg
    SET flightType = 'D' 
    WHERE source_country_code == destination_country_code
    '''
    )

cur.execute(
    '''
    UPDATE FlightLeg
    SET flightType = 'I'
    WHERE source_country_code != destination_country_code
    '''
    )

con.commit()
con.close()

### 4.1, 4.2, 4.3

In [72]:
con = sqlite3.connect('flight.db')

cur = con.cursor()

# Which plane did execute the most flights?

cur.execute(
    '''
    SELECT *,count(tailNumber) as 'number_of_flights'
    FROM FlightLeg 
    GROUP BY tailNumber 
    ORDER BY number_of_flights 
    DESC
    LIMIT 3
    '''
    )

# Displaying result of three planes to show that answer is correct.
# Two planes achieved draft in number of executed flights.

for plane in cur.fetchall():
    print(plane)

# Which plane had flown the most minutes? 

cur.execute(
    '''
    SELECT *, sum(flightDuration)
    FROM FlightLeg
    GROUP BY tailNumber
    ORDER BY sum(flightDuration) DESC 
    LIMIT 1
    '''
    )

print(cur.fetchall())

# Which flight considering domestic and national was the shortest,
# and which one the longest and how many minutes last?

cur.execute(
    '''
    SELECT tailNumber, sum(flightDuration)
    FROM FlightLeg WHERE flightType = 'I'
    GROUP BY tailNumber
    ORDER BY sum(flightDuration) DESC
    LIMIT 1
    '''
    )

print('\nThe plane which flew most minutes in international flights,\n',\
    cur.fetchall())

cur.execute(
    '''
    SELECT tailNumber, sum(flightDuration)
    FROM FlightLeg WHERE flightType = 'I'
    GROUP BY tailNumber
    ORDER BY sum(flightDuration)
    LIMIT 1
    '''
    )

print('\nThe plane which flew least minutes in international flights, \n',\
    cur.fetchall())

cur.execute(
    '''
    SELECT tailNumber, sum(flightDuration)
    FROM FlightLeg WHERE flightType = 'D'
    GROUP BY tailNumber
    ORDER BY sum(flightDuration) DESC
    LIMIT 1
    '''
    )

print('\nThe plane which flew the most minutes in domestic flights, \n',\
    cur.fetchall())


cur.execute(
    '''
    SELECT tailNumber, sum(flightDuration)
    FROM FlightLeg WHERE flightType = 'D'
    GROUP BY tailNumber
    ORDER BY sum(flightDuration)
    LIMIT 1
    '''
    )

print('\nThe plane which flew least minutes in domestic flights, \n',\
    cur.fetchall())


con.close()

(7, 'A6-ENS', 'DXB', 'ARE', 'SEZ', 'SYC', '2021-11-30 22:39:38', '2021-12-01 02:41:00', 241, 'I', 13)
(1, 'A6-ENE', 'ACC', 'GHA', 'ABJ', 'CIV', '2021-11-29 13:35:00', '2021-11-29 14:17:00', 42, 'I', 13)
(70, 'A6-ENJ', 'BEY', 'LBN', 'DXB', 'ARE', '2021-11-29 17:09:45', '2021-11-29 20:39:00', 209, 'I', 12)
[(6, 'A6-EVF', 'DXB', 'ARE', 'JFK', 'USA', '2021-11-30 22:57:00', '2021-12-01 13:01:00', 844, 'I', 3426)]

The plane which flew the most minutes in international flights, 
 [('A6-EVF', 3426)]

The plane which flew least minutes in international flights, 
 [('A6-EUR', 293)]

The plane which flew the most minutes in domestic flights, 
 [('A6-EPO', 136)]

The plane which flew least minutes in domestic flights, 
 [('A6-ECH', 61)]


###  4.4 Ile jest błędnych rekordów opisujących loty, które wskazują, że samolot wykonywał więcej niż jeden lot jednocześnie? Wyświetl wszystkie pary takich konfliktowych lotów. 

In [77]:
con = sqlite3.connect('flight.db')

cur = con.cursor()

# This is the only plane that has invalid data

cur.execute(
    '''
    WITH test_query AS 
    (SELECT *,lag(departure_time,1)
    OVER (PARTITION BY tailNumber
    ORDER BY departure_time DESC)
    last_flight_time 
    FROM  FlightLeg)
    SELECT * 
    FROM test_query 
    WHERE last_flight_time < landing_time 
    OR landing_time > last_flight_time
    '''
    )
print(cur.fetchall())


con.close()

[(499, 'A6-ECS', 'EWR', 'USA', 'ATH', 'GRC', '2021-11-30 05:14:44', '2021-11-30 15:50:00', 635, 'I', '2021-11-30 15:45:00')]


### 4.5.Odstęp pomiędzy którymi lotami tego samego dowolnego samolotu był najkrótszy i ile minut trwał? 

In [76]:
con = sqlite3.connect('flight.db')

cur = con.cursor()

# The time between flights was calculated for plane with 
# tailNumber =  A6-EBK

cur.execute(
    '''
    WITH test_query 
    AS (SELECT *,lag(departure_time,1)
    OVER (PARTITION BY tailNumber
    ORDER BY departure_time DESC)
    last_flight_time 
    FROM  FlightLeg)
    SELECT *,
    CAST(
    ROUND(((JULIANDAY(last_flight_time)- 
    JULIANDAY(landing_time)) *86400 )/60) as INT)
    as 'time_between'
    FROM test_query 
    WHERE tailNumber = 'A6-EBK'
    '''
    )

for plane in cur.fetchall():
    print(plane)

print('The least amount of time is equal to 328 minutes.')

con.close()

(985, 'A6-EBK', 'DXB', 'ARE', 'CPH', 'DNK', '2021-11-30 23:11:00', '2021-12-01 05:46:00', 395, 'I', None, None)
(969, 'A6-EBK', 'ORD', 'USA', 'DXB', 'ARE', '2021-11-29 18:25:52', '2021-11-30 07:14:00', 768, 'I', '2021-11-30 23:11:00', 957)
(141, 'A6-EBK', 'DXB', 'ARE', 'ORD', 'USA', '2021-11-28 22:40:00', '2021-11-29 12:58:00', 858, 'I', '2021-11-29 18:25:52', 328)
(467, 'A6-EBK', 'LCK', 'USA', 'DXB', 'ARE', '2021-11-27 23:24:00', '2021-11-28 12:26:00', 782, 'I', '2021-11-28 22:40:00', 614)
